In [110]:
token ='jyJkyAhgZxfZTifXwMPichXZAgoZTcrv'

    
import requests

def make_request(endpoint, payload=None):
    return requests.get(
    f"https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}",
    headers={
        'token':'jyJkyAhgZxfZTifXwMPichXZAgoZTcrv'
    },
    params=payload
    )


response = make_request('datasets')
requests.get?

In [19]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid' : 'GHCND',
        'stationid' : 'GHCND:USW00094728',
        'locationid' : 'CITY:US360019',
        'startdate' : '2018-10-01',
        'enddate' : '2018-10-31',
        'datatypeid' : ['TMIN', 'TMAX', 'TOBS'], # temperature at time of observation, min, and max
        'units' : 'metric',
        'limit' : 1000
    }
)
response.status_code

200

In [56]:
def get_item(name, what, endpoint, start=1, end=None):
    
    mid = (start + (end if end else 1)) // 2
    name = name.lower()
    
    payload = {
        'datasetid' : 'GHCND',
        'sortfield' : 'name',
        'offset' : mid, # we will change the offset each time
        'limit' : 1 # we only want one value back
    }
    
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        end = end if end else response.json()['metadata']['resultset']['count']
        print(f'start: {start} mid: {mid} end: {end}')
        
        current_name = response.json()['results'][0]['name'].lower()
        print(current_name)
        
        if name in current_name:
            return response.json()['results'][0]
        else:
            if start >= end:
                return {}
            elif name < current_name:
                return get_item(name, what, endpoint, start, mid-1)
            elif name > current_name:
                return get_item(name, what, endpoint, mid+1, end)
            
    else:
        print(f'Response not ok, status {response.status_code}')
    

In [57]:
def get_location(name):
    
    return get_item(name, {'locationcategoryid':'CITY'}, 'locations')

## Begining of testing phase

In [111]:
payload = {
        'datasetid' : 'GHCND',
        'sortfield' : 'name',
        'offset' : 1255, # we will change the offset each time
        'limit' : 1
        }

response1 = make_request('locations', {**payload, 'locationcategoryid':'CITY'}) ## how does it work - SOLVED - nested dictionary

In [85]:
response1.json()['results'][0]['name']

'New York, NY US'

In [104]:
payload = {
        'datasetid' : 'GHCND',
        'sortfield' : 'name',
        'offset' : 1255, # we will change the offset each time
        'limit' : 1
        }

def rand_print(**kwargs):
    print(kwargs)
    
rand_print(**payload)

{'datasetid': 'GHCND', 'sortfield': 'name', 'offset': 1255, 'limit': 1}


## The end of testing phase

In [113]:
nyc = get_location('new york')
nyc

start: 1 mid: 1 end: 1983
's-hertogenbosch, nl
start: 2 mid: 992 end: 1983
liege, be
start: 993 mid: 1488 end: 1983
richards bay, sf
start: 993 mid: 1240 end: 1487
neiva, co
start: 1241 mid: 1364 end: 1487
peachtree city, ga us
start: 1241 mid: 1302 end: 1363
omsk, rs
start: 1241 mid: 1271 end: 1301
nong khai, th
start: 1241 mid: 1255 end: 1270
new york, ny us


{'mindate': '1869-01-01',
 'maxdate': '2019-11-03',
 'name': 'New York, NY US',
 'datacoverage': 1,
 'id': 'CITY:US360019'}

In [114]:
nyc['id']

'CITY:US360019'

In [115]:
central_park = get_item('NY City Central Park', {'locationid' : nyc['id']}, 'stations')

start: 1 mid: 1 end: 269
albertson 0.2 sse, ny us
start: 2 mid: 135 end: 269
new hyde park 1.6 ne, ny us
start: 136 mid: 202 end: 269
saddle brook twp 0.3 nne, nj us
start: 136 mid: 168 end: 201
oyster bay, ny us
start: 136 mid: 151 end: 167
newark liberty international airport, nj us
start: 152 mid: 159 end: 167
oakland 0.2 wnw, nj us
start: 152 mid: 155 end: 158
north caldwell 0.6 sse, nj us
start: 156 mid: 157 end: 158
north wantagh 0.4 wsw, ny us
start: 158 mid: 158 end: 158
ny city central park, ny us


In [116]:
central_park

{'elevation': 42.7,
 'mindate': '1869-01-01',
 'maxdate': '2019-11-02',
 'latitude': 40.77898,
 'name': 'NY CITY CENTRAL PARK, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00094728',
 'elevationUnit': 'METERS',
 'longitude': -73.96925}

In [117]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid' : 'GHCND',
        'stationid' : 'GHCND:USW00014732',
        'locationid' : nyc['id'],
        'startdate' : '2018-10-01',
        'enddate' : '2018-10-31',
        'datatypeid' : ['TMIN', 'TMAX', 'TAVG'], # temperature at time of observation, min, and max
        'units' : 'metric',
        'limit' : 1000
    }
)
response.status_code

200

In [119]:
response.json()

{'metadata': {'resultset': {'offset': 1, 'count': 93, 'limit': 1000}},
 'results': [{'date': '2018-10-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 21.2},
  {'date': '2018-10-01T00:00:00',
   'datatype': 'TMAX',
   'station': 'GHCND:USW00014732',
   'attributes': ',,W,2400',
   'value': 25.6},
  {'date': '2018-10-01T00:00:00',
   'datatype': 'TMIN',
   'station': 'GHCND:USW00014732',
   'attributes': ',,W,2400',
   'value': 18.3},
  {'date': '2018-10-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 22.7},
  {'date': '2018-10-02T00:00:00',
   'datatype': 'TMAX',
   'station': 'GHCND:USW00014732',
   'attributes': ',,W,2400',
   'value': 26.1},
  {'date': '2018-10-02T00:00:00',
   'datatype': 'TMIN',
   'station': 'GHCND:USW00014732',
   'attributes': ',,W,2400',
   'value': 19.4},
  {'date': '2018-10-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW000

In [120]:
df = pd.DataFrame(response.json()['results'])
df.head()

,attributes,datatype,date,station,value
0,"H,,S,",TAVG,2018-10-01T00:00:00,GHCND:USW00014732,21.2
1,",,W,2400",TMAX,2018-10-01T00:00:00,GHCND:USW00014732,25.6
2,",,W,2400",TMIN,2018-10-01T00:00:00,GHCND:USW00014732,18.3
3,"H,,S,",TAVG,2018-10-02T00:00:00,GHCND:USW00014732,22.7
4,",,W,2400",TMAX,2018-10-02T00:00:00,GHCND:USW00014732,26.1


In [121]:
df.to_csv('marek_df.csv', index=False)

In [124]:
df.datatype.value_counts()

TMIN    31
TAVG    31
TMAX    31
Name: datatype, dtype: int64